In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.1 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
from matplotlib import pyplot as plt
import time
import pandas as pd
import sentencepiece as spm
from sentencepiece import SentencePieceProcessor
import os
import json
from logging import getLogger

학습된 vocab에서 토큰 임베딩 구하기
<br>
vocab모델 로딩

In [ ]:
vocab_file = "pytest.model"
#vocab = spm.SentencePieceProcessor()
#vocab.load(vocab_file)

logger = getLogger()

입력값을 임베딩 해주기

In [ ]:
class tokenizer:
  def __init__(self):
      super().__init__()
      model_path = "pytest.model"
      assert os.path.isfile(model_path), model_path
      self.sp_model = SentencePieceProcessor(model_file=model_path)
      logger.info(f"Reloaded SentencePiece model from {model_path}")
  def encode(lines):
    inputs = []
    for line in lines:
      #pieces = vocab.encode_as_pieces(line)
      ids = sp_model.encode_as_ids(line)
      inputs.append(torch.tensor(ids))
    inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
    return inputs
  def decode(lines):
    input =[]
    for line in lines:
      line=line.tolist()
      txt=sp_model.decode(line)
      input.append(txt)
    return input

In [ ]:
def encode(lines):
  inputs = []
  for line in lines:
    #pieces = vocab.encode_as_pieces(line)
    ids = vocab.encode_as_ids(line)
    inputs.append(torch.tensor(ids))
  inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
  return inputs
def decode(lines):
  input =[]
  for line in lines:
    line=line.tolist()
    txt=vocab.decode(line)
    input.append(txt)
    #vocab.decode(line)
  return input
#encode(["print(lamda x: for i in x)","for in range(len a)"])
print(encode(["print(lamda x: for i in x)","for in range(len(a))"]))
print(decode(encode(["print(lamda x: for i in x)","for in range(len(a))"])))
#decode(encode("print(lamda x: for i in x)"))

tensor([[  32, 7934,  128, 7931,  298,  130, 7941,   65,   17,   28,  130, 7935],
        [  65,   28,  143, 7934,  103, 7934, 7925,   72,    0,    0,    0,    0]])
['print(lamda x: for i in x)', 'for in range(len(a))']


트렌스포머 설정 로딩

In [ ]:
""" configuration json을 읽어들이는 class """
class Config(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)
mconfig=Config.load("config.json")

학습 데이터로 부터 베치 만들기

rmsnorm

In [ ]:
class RMSNorm(nn.Module):
    def __init__(self, layer_shape, eps=1e-8, bias=False):
        super(RMSNorm, self).__init__()
        self.register_parameter("scale", nn.Parameter(torch.ones(layer_shape)))

    def forward(self, x):
        """
        assumes shape is (batch, seq_len, d_model)
        """
        # frob norm is not the same as RMS. RMS = 1/sqrt(N) * frob norm
        ff_rms = torch.linalg.norm(x, dim=(1,2)) * x[0].numel() ** -.5
        raw = x / ff_rms.unsqueeze(-1).unsqueeze(-1)
        return self.scale[:x.shape[1], :].unsqueeze(0) * raw


swiglu

In [ ]:
class SwiGLU(nn.Module):
    """
    Swish-Gated Linear Unit
    https://arxiv.org/pdf/2002.05202v1.pdf
    """
    def __init__(self, size):
        super().__init__()
        self.config = config
        self.linear_gate = nn.Linear(size, size)
        self.linear = nn.Linear(size, size)
        self.beta = torch.randn(1, requires_grad=True)

        self.beta = nn.Parameter(torch.ones(1))
        self.register_parameter("beta", self.beta)

    def forward(self, x):
        swish_gate = self.linear_gate(x) * torch.sigmoid(self.beta * self.linear_gate(x))
        out = swish_gate * self.linear(x)
        return out

rope

In [ ]:
def get_rotary_matrix(context_window, embedding_dim):
    R = torch.zeros((context_window, embedding_dim, embedding_dim), requires_grad=False)
    for position in range(context_window):
        for i in range(embedding_dim//2):
            theta = 10000. ** (-2.*(i - 1) / embedding_dim)
            m_theta = position * theta
            R[position, 2*i,2*i] = np.cos(m_theta)
            R[position, 2*i,2*i+1] = - np.sin(m_theta)
            R[position, 2*i+1,2*i] = np.sin(m_theta)
            R[position, 2*i+1,2*i+1] = np.cos(m_theta)
    return R
class RoPEMaskedAttentionHead(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.w_q = nn.Linear(config['d_model'], config['d_model'], bias=False)
        self.w_k = nn.Linear(config['d_model'], config['d_model'], bias=False)
        self.w_v = nn.Linear(config['d_model'], config['d_model'], bias=False)

        self.R = get_rotary_matrix(config['context_window'], config['d_model'])

    def get_rotary_matrix(context_window, embedding_dim):
        R = torch.zeros((context_window, embedding_dim, embedding_dim), requires_grad=False)
        for position in range(context_window):
            for i in range(embedding_dim//2):
                theta = 10000. ** (-2.*(i - 1) / embedding_dim)
                m_theta = position * theta
                R[position, 2*i,2*i] = np.cos(m_theta)
                R[position, 2*i,2*i+1] = - np.sin(m_theta)
                R[position, 2*i+1,2*i] = np.sin(m_theta)
                R[position, 2*i+1,2*i+1] = np.cos(m_theta)
        return R

    def forward(self, x, return_attn_weights=False):
        b,m,d = x.shape

        q = self.w_q(x)
        k = self.w_k(x)
        v = self.w_v(x)

        q_rotated = (torch.bmm(q.transpose(0,1), self.R[:m])).transpose(0,1)
        k_rotated = (torch.bmm(k.transpose(0,1), self.R[:m])).transpose(0,1)

        activations = F.scaled_dot_product_attention(
            q_rotated,k_rotated,v,dropout_p =.1, is_causal=True
        )

        if return_attn_weights:
            attn_mask = torch.tril(torch.ones((m,m)), diagonal=0)
            attn_weights = torch.bmm(q_rotated, k_rotated.transpose(1,2)) / np.sqrt(d) + attn_mask
            attn_weights = F.softmax(attn_weights, dim=-1)
            return activations, attn_weights
        return activations

In [ ]:
class myblock(nn.module):
  def __init__(self,config):
      super().__init__()
      self.config=config
      self.rms = RMSNorm((config['context_window'], config['d_model']))
      self.attention = RoPEMaskedMultiheadAttention(config)
      self.feedforward = nn.Sequential(
            nn.Linear(config['d_model'], config['d_model']),
            SwiGLU(config['d_model']),
        )
  def forward(self, x):
        x = self.rms(x) # rms pre-normalization
        x = x + self.attention(x)

        x = self.rms(x) # rms pre-normalization
        x = x + self.feedforward(x)
        return x

학습실행코드 생성

In [ ]:

def train(model, optimizer, scheduler=None, config=MASTER_CONFIG, print_logs=False):
    losses = []
    start_time = time.time()
    for epoch in range(config['epochs']):
        optimizer.zero_grad()

        xs, ys = get_batches(dataset, 'train', config['batch_size'], config['context_window'])
        logits, loss = model(xs, targets=ys)
        loss.backward()
        optimizer.step()

        if scheduler:
            scheduler.step()

        if epoch % config['log_interval'] == 0:
            batch_time = time.time() - start_time
            x = evaluate_loss(model)
            losses += [x]
            if print_logs:
                print(f"Epoch {epoch} | val loss {x['val']:.3f} | Time {batch_time:.3f} | ETA in seconds {batch_time * (config['epochs'] - epoch)/config['log_interval'] :.3f}")
            start_time = time.time()

            if scheduler:
                print("lr: ", scheduler.get_lr())

    print("validation loss: ", losses[-1]['val'])
    return pd.DataFrame(losses).plot()

학습한 토크나이저를 이용해서 만든 학습용 파일에서 크기 맞춰서 입력

In [ ]:
def get_batches(split, batch_size, config=MASTER_CONFIG):
  if split == 'val':
    val=open("val.json",'r')
    batch_data = val
  elif split == 'test':
    test=open("test.json",'r')
    batch_data = test
  else:
    train=open("train.json",'r')
    batch_data = train
  d=json.load(batch_data)
  id_dict = {item["id"]: item for item in d}
  max_id = max(id_dict.keys()) if id_dict else None
  rand= random.randint(0, max_id-batch_size)
  x=[]
  y=[]
  for _ in range(rand,rand+batch_size):
    ans,q=searchbyid(_,id_dict)
    x.append(ans)
    y.append(q)
    rand=rand+1
  x = torch.nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=0)
  y = torch.nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=0)
  return x,y


배치를 가져와서 모델 평가하는 코드

In [ ]:
@torch.no_grad()  # don't compute gradients for this function
def evaluate_loss(model, config=MASTER_CONFIG):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = []
        for _ in range(10):
            xb, yb = get_batches(split, config['batch_size'])
            _, loss = model(xb, yb)
            losses.append(loss.item())
        out[split] = np.mean(losses)
    model.train()
    return out